# explore how OMF applications interact with OSIsoft Cloud Services

Topic, Subscription creation and related configuration

two scenarios
1. OMF application can create and write to types and streams
2. OMF application can only write to specific streams

In [1]:
# specify a unique prefix to create objects, suggestion use your nickname!
example_prefix = 'oak'

In [2]:
!pip3 install ocs_sample_library_preview

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
# setup the environment, using statements defined in a python script file
# this requires a config.ini file configured for your OCS environment
%run ocs_setup.py

# Create a second object to access OCS to query routes that are not supported in the OCS sample Python library
# use previously defined config object to use the same configuration information
baseClient = BaseClient("v1", config.get('Access', 'Tenant'), config.get('Access', 'Resource'), 
                        config.get('Credentials', 'ClientId'), config.get('Credentials', 'ClientSecret'))

# Customize namespace_id if reqeuired
#namespace_id = "DerekETesting"

# define the uri prefix for the namespace
baseUri = f'{baseClient.uri_API}/Tenants/{baseClient.tenant}/Namespaces/{namespace_id}'

# verify everything looks correct!
print(f'''
url: {ocsClient.uri}
tenant: {ocsClient.tenant}
namespace: {namespace_id}
''')

Using OSIsoft OCS sample library

url: https://staging.osipi.com
tenant: efe27258-f6d5-4ea6-a001-3e4a82777710
namespace: 8200557b-8fa9-4b4f-a884-92967c79251f



# Configuration details for the OMF application

In [4]:
#
# Set configuration information to create topic and subscription
# Unless customizing, for the purposes of this example, no changes are required
#

# topic associates a client - used by the OMF application with OCS
conf_topic = {
  "Name": f"{example_prefix}.omf.app.write",
  "Description": "create and write objects",
  "ClientIds": [f"{example_prefix}.devices"]}
  
# subscription associates a topic with a namespace
# there can be multiple subscriptions per topic
conf_subscription = { 
  "Name": f"{example_prefix}.omf.write",
  "Description": "create and write objects",
  "NamespaceId": namespace_id,
  "TenantId": ocsClient.tenant,
  "TopicNamespaceId": namespace_id,
  "TopicTenantId": ocsClient.tenant,
  "Owner": f"{example_prefix}.omf.write",
  "Role": f"{example_prefix}.omf.write"
}

# subscription that will only have write access to specific streams
conf_subscription_restricted = conf_subscription.copy()
conf_subscription_restricted.update({ 
  "Name": f"{example_prefix}.omf.write.restricted",
  "Description": "write to specific objects only",
  "Owner": f"{example_prefix}.omf.write.restricted",
  "Role": f"{example_prefix}.omf.write.restricted"
})

In [5]:
# review configuration settings
conf_topic
conf_subscription
conf_subscription_restricted

{'Name': 'oak.omf.app.write',
 'Description': 'create and write objects',
 'ClientIds': ['oak.devices']}

{'Name': 'oak.omf.write',
 'Description': 'create and write objects',
 'NamespaceId': '8200557b-8fa9-4b4f-a884-92967c79251f',
 'TenantId': 'efe27258-f6d5-4ea6-a001-3e4a82777710',
 'TopicNamespaceId': '8200557b-8fa9-4b4f-a884-92967c79251f',
 'TopicTenantId': 'efe27258-f6d5-4ea6-a001-3e4a82777710',
 'Owner': 'oak.omf.write',
 'Role': 'oak.omf.write'}

{'Name': 'oak.omf.write.restricted',
 'Description': 'write to specific objects only',
 'NamespaceId': '8200557b-8fa9-4b4f-a884-92967c79251f',
 'TenantId': 'efe27258-f6d5-4ea6-a001-3e4a82777710',
 'TopicNamespaceId': '8200557b-8fa9-4b4f-a884-92967c79251f',
 'TopicTenantId': 'efe27258-f6d5-4ea6-a001-3e4a82777710',
 'Owner': 'oak.omf.write.restricted',
 'Role': 'oak.omf.write.restricted'}

# Query types and streams - verify we have access

In [ ]:
# query types
for ocs_type in ocsClient.Types.getTypes(namespace_id,query="*",count=200):
    print(f'id: {ocs_type.Id:40} name: {ocs_type.Name}')

In [ ]:
# display stream events
for stream in ocsClient.Streams.getStreams(namespace_id,query="*"):
    print(stream.Name)
    print(ocsClient.Streams.getLastValue(namespace_id,stream.Id))

# helper functions - simplify OCS access

Ideally this code would be in a library - idea for the future

In [8]:
# construct objects to enable queries to OCS that are not supported by the OCS sample Python library

import requests
baseUri = None

def requestOCS(path,verb=None,payload=None,http_code=False,debug=False):
    """
    query data from OCS REST API - useful where OCS Python library does not support an API route or feature
    :param path: uri suffix of API call to append to baseUri parameter (which is usually a route to the namespace)
    :param verb: HTTP verb, default: get
    :param payload: for POST, PUT requests
    :param http_code: print http status code
    :param debug: display route 
    """
    baseUri = f'{baseClient.uri_API}/Tenants/{baseClient.tenant}/Namespaces/{namespace_id}'
    if debug:
        baseClient.AcceptVerbosity = True
        print(f'route {baseUri}{path}')
    if verb is None or verb.lower() == "get":
        baseClient.AcceptVerbosity = True
        response = requests.get(
                f'{baseUri}{path}',
        headers=baseClient.sdsHeaders())
    elif verb.lower() == "post":
        response = requests.post(
                f'{baseUri}{path}',
                data=payload,
        headers=baseClient.sdsHeaders())
    elif verb.lower() == "put":
        response = requests.put(
                f'{baseUri}{path}',
                data=payload,
        headers=baseClient.sdsHeaders())
    elif verb.lower() == "delete":
        response = requests.delete(
                f'{baseUri}{path}',
        headers=baseClient.sdsHeaders())
    else:
        print(f"can't grok that")
        return
    if http_code:
        print(f'http code: {response.status_code}')
    baseClient.checkResponse(
            response, f"Query failed."
    )
    if response.content is not None:
        try:
            content = json.loads(response.content.decode('utf-8'))
            #print(content)
            return content
        except:
            print(response.text)
    if response.status_code < 200 or response.status_code >= 300:
        print("an error!")
    response.close()
    
# load lookup tables for users, roles and clients to augment displaying objects that show GUIDs
def cache():
    global users,clients,roles
    users = {}
    clients =  {}
    roles = {}
    user_count = 400 # guess a number larger than # of clients, do this better...
    response = requestOCS(f'/../../users?count={user_count}',"get")
    for user in response:
        #print(f"{client['ClientId']:20},{client['Name']}")
        users[user['Id']] = user['Name']
    if len(users) == 400:
        print("not all users retrieved, update user_count")
    clients = {}
    client_count = 400 # guess a number larger than # of clients, do this better...
    response = requestOCS(f'/../../clientcredentialclients?count={client_count}',"get")
    for client in response:
        #print(f"{client['Id']:20},{client['Name']}")
        clients[client['Id']] = client['Name']

    roles = {}
    response = requestOCS(f'/../../roles',"get")
    for role in response:
    #    print(f"{role['Id']:20},{role['Name']}")
        roles[role['Id']] = role['Name']
    
# given a acl, replace id with name
def lookup_ace_id(response):
    """ lookup_id
    """
    for x in response['RoleTrusteeAccessControlEntries']:
        object_name = x['Trustee']['ObjectId']
        try:
            if clients[object_name]:
                 x['Trustee']['ObjectId'] = clients[object_name]
        except:
            pass
        try:
            if roles[object_name]:
                 x['Trustee']['ObjectId'] = roles[object_name]
        except:
            pass
    return response

# given a list object replace client id with name
def lookup_client_id(response):
    """ lookup_clientid
    """
    if isinstance(response, list):
        for record in response:
            client_object = []
            for client in record['ClientIds']:
                client_object.append(clients[client])
            if client_object is not None:
                record['ClientIds'] = client_object
    elif isinstance(response,str):
        response = clients[response]
    return response

# try to find a matching name for given id
def lookup_id(id):
    """ lookup_clientid
    """
    if isinstance(id,str):
        try:
            if clients[id]:
                id = clients[id]
        except:
            pass
        try:
            if roles[id]:
                id = roles[id]
        except:
            pass
        try:
            if users[id]:
                id = users[id]
        except:
            pass
    if isinstance(id,dict):
        try:
            if clients[id['ObjectId']]:
                id['ObjectId'] = clients[id['ObjectId']]
        except:
            pass
        try:
            if roles[id['ObjectId']]:
                id['ObjectId'] = roles[id['ObjectId']]
        except:
            pass
        try:
            if users[id['ObjectId']]:
                id['ObjectId'] = users[id['ObjectId']]
        except:
            pass
    return id

# load the cache
cache()

# Check if objects exist
if objects exist either change the configuration settings or delete objects, see also, delete examples in this notebook

In [9]:
# print topic if it exists based upon configuration
response = requestOCS(f"/Topics")
obj = [obj for obj in response if obj['Name'] == conf_topic['Name']]
pprint.pprint(lookup_client_id(obj)) if obj else print("topic does not exist")
if obj:
    # print topic client if it exists
    for client in obj[0]['ClientIds']:
        #print(client)
        _ = [("topic client: ",key,value) for key,value in clients.items() if key == client]

# ... for subscription
response = requestOCS(f"/Subscriptions")
obj = [obj for obj in response if obj['Name'] == conf_subscription['Name']]
pprint.pprint(obj) if obj else print("subscription does not exist")
if obj:
    # display subscription owner
    response = requestOCS(f"/Subscriptions/{obj[0]['Id']}/owner")
    pprint.pprint(lookup_id(response))
    _ = [("user: ",key,value) for key,value in users.items() if key == response['ObjectId']]
    _ = [("client: ",key,value) for key,value in clients.items() if key == response['ObjectId']]

# for configuration topic clients
for client in conf_topic['ClientIds']:
    _ = [("topic client: ",key,value) for key,value in clients.items() if value == client]
if not _:
    print("topic ClientIds do not exist")
if not [print(key,value) for key,value in clients.items() if value == conf_subscription['Owner']]:
    print("subscription owner does not exist") 

for role in conf_subscription['Role'],conf_subscription['Role']:
    _ = [("role: ",key,value) for key,value in roles.items() if value == role]
if not _:
    print("roles do not exist")

topic does not exist
subscription does not exist
topic ClientIds do not exist
subscription owner does not exist
roles do not exist


# Create client for topic, topic and subscription

In [10]:
# create a client to be used by OMF application(s)

# get id for Account Member
ocs_account = [key for key,value in roles.items() if value == "Account Member"]
ocs_client = []
for topic_client in conf_topic['ClientIds']:
    ocs_client.append({
        "RoleIds": [
            f'{ocs_account[0]}'
        ],
        "Name": f'{topic_client}',        
        "Enabled": True
    })

topic_clients = {}
for topic_client in ocs_client:
    response = requestOCS("/../../ClientCredentialClients/",verb="post",payload=json.dumps(topic_client),http_code=True)
    topic_clients[response['Client']['Name']] = { "id": f"{response['Client']['Id']}", "secret" : f"{response['Secret']}" }

http code: 201


In [11]:
# create a topic
# todo: add all clients
topic_obj = { 'ClientIds': [f"{topic_clients[conf_topic['ClientIds'][0]]['id']}"],
              'Description': f"{conf_topic['Description']}",
              'Name': f"{conf_topic['Name']}"}

#pprint.pprint(topic_obj)
topic = requestOCS("/Topics/","post",payload=json.dumps(topic_obj),http_code=True)

http code: 200


In [12]:
# show the topic
topic

{'Id': '026b5e4e-2f92-44b0-8fbf-b717895b12fc',
 'Name': 'oak.omf.app.write',
 'Description': 'create and write objects',
 'CreatedDate': '2019-10-17T01:01:01.2953302Z',
 'ClientIds': ['444d29af-e82e-4224-b2ee-edbc9412115a']}

In [13]:
# create a subscription

sub_obj = {'Name': conf_subscription['Name'],
          'TopicId': topic['Id'],
          'TopicTenantId': ocsClient.tenant,
          'TopicNamespaceId': namespace_id,
          'TenantId': ocsClient.tenant,
          'NamespaceId': namespace_id,
          'Description': conf_subscription['Description'],
          'Type': 1,
          'Enabled': True}

subscription = requestOCS("/Subscriptions","post",payload=json.dumps(sub_obj),http_code=True)

http code: 200


In [14]:
# show the subscription
subscription

{'Id': '8976c37b-2226-4940-8b1e-ecf32f011df8',
 'Name': 'oak.omf.write',
 'TopicId': '026b5e4e-2f92-44b0-8fbf-b717895b12fc',
 'TopicTenantId': 'efe27258-f6d5-4ea6-a001-3e4a82777710',
 'TopicNamespaceId': '8200557b-8fa9-4b4f-a884-92967c79251f',
 'TenantId': 'efe27258-f6d5-4ea6-a001-3e4a82777710',
 'NamespaceId': '8200557b-8fa9-4b4f-a884-92967c79251f',
 'Description': 'create and write objects',
 'Type': 1,
 'CreatedDate': '2019-10-17T01:01:04.5303963Z',
 'Enabled': True}

# create a role, client and set client to subscription owner

In [15]:
# create the role
role_obj = {
        "Name": f"{conf_subscription['Role']}",
        "Description": f"{conf_subscription['Description']}",
        "RoleScope": 1
    }
role = requestOCS("/../../Roles/",verb="post",payload=json.dumps(role_obj),http_code=True)

http code: 201


In [16]:
role

{'Id': 'c1682e0d-9383-4344-ac3a-c344598f341f',
 'Name': 'oak.omf.write',
 'Description': 'create and write objects',
 'RoleScope': 1}

In [17]:
# create the client

client_obj = { "RoleIds": [
                    role['Id'],
                    f"{[key for key,value in roles.items() if value == 'Account Member'][0]}"
                ],
                "Name": f"{conf_subscription['Owner']}",
                "Description": f"{conf_subscription['Description']}",
                "Enabled": True
             }

#print.pprint(client_obj)
client = requestOCS("/../../ClientCredentialClients/",verb="post",payload=json.dumps(client_obj),http_code=True)

http code: 201


In [18]:
client

{'Secret': 'rVBDV5R+D7Pnzatac14gVGCJ5JvZUqNfKrRAadqVjE4=',
 'Id': 30126,
 'Description': None,
 'ExpirationDate': None,
 'Client': {'RoleIds': ['c1682e0d-9383-4344-ac3a-c344598f341f',
   'd1cf7ef9-02ac-4322-bb5a-1a22766203f4'],
  'Id': 'efcdc912-fd37-4c00-b055-b4eb3b3ab214',
  'Name': 'oak.omf.write',
  'Enabled': True,
  'AccessTokenLifetime': 3600,
  'Tags': []}}

In [19]:
# change subscription owner

owner_obj = {
	"Type": 2,
	"ObjectId": client['Client']['Id'],
    "TenantId": ocsClient.tenant
}

#pprint.pprint(owner_obj)
owner = requestOCS(f"/Subscriptions/{subscription['Id']}/owner","put",payload=json.dumps(owner_obj),http_code=True)

http code: 200


In [23]:
cache() # refresh to add newly created client
lookup_id(owner)

{'Type': 2,
 'ObjectId': 'oak.omf.write',
 'TenantId': 'efe27258-f6d5-4ea6-a001-3e4a82777710'}

# Add subscription owner role to default acl for types and streams

In [24]:
# add the client role to access control entry for types and streams *if* it doesn't already exist
# (Note: if it fails, run a second time - appears to be notebook related)

# Check if the client already exists in the acl
for object in ("types","streams"):
    acl = requestOCS(f'/AccessControl/{object}',"get")
    trustee =  [ trustee for trustee in acl['RoleTrusteeAccessControlEntries'] if trustee['Trustee']['ObjectId'] == role['Id']]
    # if not, add it!
    if not trustee:
        print(f'adding acl for {object}....')
        #pprint.pprint(acl)
        trustee_obj = { 'Trustee': { 'Type': 3, 
                     'ObjectId': role['Id'] },
                     'AccessRights': 2}
        acl['RoleTrusteeAccessControlEntries'].append(trustee_obj)
        #pprint.pprint(acl)
        response = requestOCS(f'/AccessControl/{object}',"put",payload=json.dumps(acl),http_code=True)
    else:
        print("role already exists, not adding...")
    # display the existing or updated acl
    lookup_ace_id(acl)

adding....
http code: 200



{'RoleTrusteeAccessControlEntries': [{'Trustee': {'Type': 3,
    'ObjectId': 'Account Member',
    'TenantId': None},
   'AccessType': 0,
   'AccessRights': 1},
  {'Trustee': {'Type': 3,
    'ObjectId': 'Account Administrator',
    'TenantId': None},
   'AccessType': 0,
   'AccessRights': 15},
  {'Trustee': {'Type': 3, 'ObjectId': 'Account Contributor', 'TenantId': None},
   'AccessType': 0,
   'AccessRights': 3},
  {'Trustee': {'Type': 3,
    'ObjectId': 'OmfHealthNoPermissions',
    'TenantId': None},
   'AccessType': 0,
   'AccessRights': 0},
  {'Trustee': {'Type': 3, 'ObjectId': 'site.omf.write', 'TenantId': None},
   'AccessType': 0,
   'AccessRights': 2},
  {'Trustee': {'Type': 3, 'ObjectId': 'oak.omf.write'}, 'AccessRights': 2}]}

adding....
http code: 200



{'RoleTrusteeAccessControlEntries': [{'Trustee': {'Type': 3,
    'ObjectId': 'Account Member',
    'TenantId': None},
   'AccessType': 0,
   'AccessRights': 1},
  {'Trustee': {'Type': 3,
    'ObjectId': 'Account Administrator',
    'TenantId': None},
   'AccessType': 0,
   'AccessRights': 15},
  {'Trustee': {'Type': 3, 'ObjectId': 'Account Contributor', 'TenantId': None},
   'AccessType': 0,
   'AccessRights': 3},
  {'Trustee': {'Type': 3, 'ObjectId': 'site.omf.write', 'TenantId': None},
   'AccessType': 0,
   'AccessRights': 3},
  {'Trustee': {'Type': 3, 'ObjectId': 'oak.omf.write'}, 'AccessRights': 2}]}

# Send OMF payload(s) to OCS

In [25]:
# create a config.ini file for use with omfDemo.py or other OMF sample script(s)
# Note: uncomment the final line to create/update the file

config = f"""
; config.ini - define OSIsoft Message Format endpoint
; update required endpoint section and comment other sections
;
;--------------------------------------------------------------
; OSIsoft Cloud Services
;
[Configurations]
Namespace = {namespace_id}
OMFVersion = 1.1

[Access]
;Resource = https://dat-b.osisoft.com
Resource = https://staging.osipi.com
Tenant = {ocsClient.tenant}
ApiVersion = v1

[Credentials]
ClientId = {topic_clients[conf_topic['ClientIds'][0]]['id']}
ClientSecret = {topic_clients[conf_topic['ClientIds'][0]]['secret']}
;"""

# Modify the location of the file if required to match the location of the omf script(s)
from pathlib import Path
config_path = Path("ocs\omf\config-omf.ini")
print(config,  file=open(config_path, 'w'))

In [26]:
# Example using omfDemo.py from https://github.com/mofff/OCS

from pathlib import Path
script = Path("ocs\omf\omfDemo.py")
config_path = Path("ocs\omf\config-omf.ini")
datadir = Path("ocs\omf\data")
cwd = Path(".").resolve()

print(f'''
Run the following command from:
{cwd}
python {script} -c {config_path} -d {datadir}
''')


Run the following command from:
C:\Users\gmoffett\Documents\working\ocs
python ocs\omf\omfDemo.py -c ocs\omf\config-omf.ini -d ocs\omf\data



# check created values

In [30]:
# query is based upon OMF payload, modify to match if required
[sds_type.Id for sds_type in ocsClient.Types.getTypes(namespace_id,query=f"id:stream-string or id:gps-coords-dd or id:stream-number")]
[ocsClient.Streams.getLastValue(namespace_id,stream.Id)  for stream in ocsClient.Streams.getStreams(namespace_id,query="Asset2*")]

['stream-string', 'gps-coords-dd', 'stream-number']

[{'IndexedDateTime': '2019-10-17T01:08:04Z', 'value': 'Ok'},
 {'IndexedDateTime': '2019-10-17T01:08:04Z',
  'latitude': '40.446° N',
  'longitude': '79.982° W'},
 {'IndexedDateTime': '2019-10-17T01:08:04Z', 'value': 300.0}]

# create  restricted access role and assign to client

In [31]:
# create the role
role_obj = {
        "Name": f"{conf_subscription_restricted['Role']}",
        "Description": f"{conf_subscription_restricted['Description']}",
        "RoleScope": 1
    }
role = requestOCS("/../../Roles/",verb="post",payload=json.dumps(role_obj),http_code=True)

http code: 201


In [32]:
# create the client
client_obj = { "RoleIds": [
                    role['Id'],
                    f"{[key for key,value in roles.items() if value == 'Account Member'][0]}"
                ],
                "Name": f"{conf_subscription_restricted['Owner']}",
                "Description": f"{conf_subscription_restricted['Description']}",
                "Enabled": True
             }

#print.pprint(client_obj)
client = requestOCS("/../../ClientCredentialClients/",verb="post",payload=json.dumps(client_obj),http_code=True)

http code: 201


In [33]:
# update cache for user, client and role details
cache()

In [74]:
# add the role to the ace for one of the streams OMF application is sending
role = {'Id': '2114bf6a-2f8d-4f7f-a147-3bb032ac9f0d'}
path = "/streams/Asset2.Temperature/accesscontrol"
ace = requestOCS(path,debug=True)
trustee = [ trustee for trustee in ace['RoleTrusteeAccessControlEntries'] if trustee['Trustee']['ObjectId'] == role['Id']]
if not trustee:
    if role is None:
        print("cannot find role...")
        exit(1)
    # create a trustee object with read/write for the created client
    trustee_obj = { 'Trustee': { 
                        'Type': 3, 
                        'ObjectId': role['Id']
                      },
                     'AccessRights': 2,
                     'AccessType': 0
                  }
    print("adding....")
    ace['RoleTrusteeAccessControlEntries'].append(trustee_obj)
    response = requestOCS(path,"put",payload=json.dumps(ace),http_code=True)
pprint.pprint(lookup_ace_id(ace))

route https://staging.osipi.com/api/v1/Tenants/efe27258-f6d5-4ea6-a001-3e4a82777710/Namespaces/8200557b-8fa9-4b4f-a884-92967c79251f/streams/Asset2.Temperature/accesscontrol
adding....
http code: 204

{'RoleTrusteeAccessControlEntries': [{'AccessRights': 1,
                                      'AccessType': 0,
                                      'Trustee': {'ObjectId': 'Account Member',
                                                  'TenantId': None,
                                                  'Type': 3}},
                                     {'AccessRights': 15,
                                      'AccessType': 0,
                                      'Trustee': {'ObjectId': 'Account '
                                                              'Administrator',
                                                  'TenantId': None,
                                                  'Type': 3}},
                                     {'AccessRights': 3,
                        

# Modify subscription owner

In [51]:
# change subscription owner
owner_obj = {
	"Type": 2,
    "ObjectId": client['Client']['Id'],
    "TenantId": ocsClient.tenant
}
owner = requestOCS(f"/Subscriptions/{subscription['Id']}/owner","put",payload=json.dumps(owner_obj),http_code=True)

http code: 200


In [63]:
lookup_id(requestOCS(f"/Subscriptions/{subscription['Id']}/owner"))

{'Type': 2,
 'ObjectId': 'oak.omf.write.restricted',
 'TenantId': 'efe27258-f6d5-4ea6-a001-3e4a82777710'}

# send OMF payload

In [52]:
# Modify the location of the file if required to match the location of the omf script(s)
config_path = Path("ocs\omf\config-omf.ini")
script = Path("ocs\omf\omfDemo.py")
datadir = Path("ocs\omf\data")
cwd = Path(".").resolve()
print(config,  file=open(config_path, 'w'))
print(f'''
Run the following command from:
{cwd}
python {script} -c {config_path} -d {datadir}
''')


Run the following command from:
C:\Users\gmoffett\Documents\working\ocs
python ocs\omf\omfDemo.py -c ocs\omf\config-omf.ini -d ocs\omf\data



# check for stream value update

In [75]:
# query is based upon OMF payload and modified stream access
# only the modified stream acl access should have an updated event
[ocsClient.Streams.getLastValue(namespace_id,stream.Id)  for stream in ocsClient.Streams.getStreams(namespace_id,query="Asset2*")]

[{'IndexedDateTime': '2019-10-17T01:08:04Z', 'value': 'Ok'},
 {'IndexedDateTime': '2019-10-17T01:08:04Z',
  'latitude': '40.446° N',
  'longitude': '79.982° W'},
 {'IndexedDateTime': '2019-10-17T01:59:19Z', 'value': 300.0}]

# Summary

In [84]:
# print topic based upon configuration
print("Topic")
response = requestOCS(f"/Topics")
obj = [obj for obj in response if obj['Name'] == conf_topic['Name']]
pprint.pprint(lookup_client_id(obj)) if obj else print("topic does not exist")
if obj:
    # print topic client if it exists
    for client in obj[0]['ClientIds']:
        #print(client)
        _ = [("topic client: ",key,value) for key,value in clients.items() if key == client]

# ... for subscription
print("Subscription")
response = requestOCS(f"/Subscriptions")
obj = [obj for obj in response if obj['Name'] == conf_subscription['Name']]
pprint.pprint(obj)
if obj:
    # display subscription owner
    print("Subscription owner")
    response = requestOCS(f"/Subscriptions/{obj[0]['Id']}/owner")
    pprint.pprint(lookup_id(response))

Topic
[{'ClientIds': ['oak.devices'],
  'CreatedDate': '2019-10-17T01:01:01.2953302',
  'Description': 'create and write objects',
  'Id': '026b5e4e-2f92-44b0-8fbf-b717895b12fc',
  'Name': 'oak.omf.app.write'}]
Subscription
[{'CreatedDate': '2019-10-17T01:01:04.5303963',
  'Description': 'create and write objects',
  'Enabled': True,
  'Id': '8976c37b-2226-4940-8b1e-ecf32f011df8',
  'Name': 'oak.omf.write',
  'NamespaceId': '8200557b-8fa9-4b4f-a884-92967c79251f',
  'TenantId': 'efe27258-f6d5-4ea6-a001-3e4a82777710',
  'TopicId': '026b5e4e-2f92-44b0-8fbf-b717895b12fc',
  'TopicNamespaceId': '8200557b-8fa9-4b4f-a884-92967c79251f',
  'TopicTenantId': 'efe27258-f6d5-4ea6-a001-3e4a82777710',
  'Type': 1}]
Subscription owner
{'ObjectId': 'oak.omf.write.restricted',
 'TenantId': 'efe27258-f6d5-4ea6-a001-3e4a82777710',
 'Type': 2}


# clean-up removing created objects

In [68]:
# Clean-up - delete stream and stream type created by OMF script

def cleanup(queries,remove=False):
    for query in queries:
        for stream in ocsClient.Streams.getStreams(namespace_id,f'name:{query}'):
            print(f'stream id: {stream.Id}, stream name: {stream.Name}')
            if remove:
                try:
                    ocsClient.Streams.deleteStream(namespace_id,stream.Id)
                except:
                    pass
            for type in ocsClient.Types.getTypes(namespace_id,query=stream.TypeId):
                print(f'type id: {type.Id}, type name: {type.Name}')
                if remove:
                    try:
                        ocsClient.Types.deleteType(namespace_id,type.Id)
                        ocsClient.Types.deleteType(namespace_id,type.TypeId)
                    except:
                        pass
        
# cleanup - disabled, i.e: False and commented:
cleanup(["Asset2.Location","Asset2.Temperature","Asset2.Status"],False)

stream id: Asset2.Location, stream name: Asset2.Location
type id: gps-coords-dd, type name: 
stream id: Asset2.Temperature, stream name: Asset2.Temperature
type id: stream-number, type name: 
stream id: Asset2.Status, stream name: Asset2.Status
type id: stream-string, type name: 


In [61]:
# Clean-up environment
# update default acl for type and streams to remove 
# print topic if it exists based upon configuration

def delete_acl(path,role,remove=False):
    """
    object: route path entry of object to remove role, e.g.: type or stream
    role: role name to remove
    remove: boolean, False to list what would be deleted
    """
    path_prefix = "/accesscontrol/"
    # lookup the role id, required to lookup role in acl
    role_id = [key for key,value in roles.items() if value == role]
    if not len(role_id) == 1:
            print(f"less or more than one role id for {role}, not supported, delete manually")
            return 1
    acl = requestOCS(f"{path_prefix}{path}")
    # update the acl removing the role accessrights if it exists
    count_acl = len(acl['RoleTrusteeAccessControlEntries'])
    trustee = acl['RoleTrusteeAccessControlEntries']
    trustee[:] = [trustee for trustee in acl['RoleTrusteeAccessControlEntries'] if trustee['Trustee']['ObjectId'] != role_id[0]]
    # update acl if required
    if count_acl > len(acl['RoleTrusteeAccessControlEntries']):
        route = f"{path_prefix}{path}"
        print(f"update {remove} for {role} acl, route: {route}")
        if remove:
              pass
              response = requestOCS(route,"put",json.dumps(acl),http_code=True)

def delete_object(path,name,remove=False):
    """
    path: partial route from namespace to object
    name: name of the object to delete
    remove: boolean, False to list what would be deleted
    """
    # get a list of the objects
    response = requestOCS(f"/{path}")
    # find the object based on the supplied name
    for entry in [obj for obj in response if obj['Name'] == name]:
        route = f"/{path}/{entry['Id']}"
        print(f"delete {remove} for {entry['Name']}, route: {route}")
        if remove:
            pass
            response = requestOCS(route,"delete",http_code=True)

# Set to true to delete
are_you_sure = False
delete_object("Subscriptions",conf_subscription['Name'],are_you_sure)
delete_object("Topics",conf_topic['Name'],are_you_sure)
_ = [delete_object("../../ClientCredentialClients",obj,are_you_sure) for obj in conf_topic['ClientIds']]
delete_acl("Types",conf_subscription['Role'],are_you_sure)
delete_acl("Streams",conf_subscription['Role'],are_you_sure)
delete_object("../../Roles",conf_subscription['Role'],are_you_sure)
delete_object("../../ClientCredentialClients",conf_subscription['Owner'],are_you_sure)
delete_object("../../ClientCredentialClients",conf_subscription_restricted['Owner'],are_you_sure)
delete_object("../../Roles",conf_subscription_restricted['Role'],are_you_sure)
print("fini")

delete False for oak.omf.write, route: /Subscriptions/8976c37b-2226-4940-8b1e-ecf32f011df8
delete False for oak.omf.app.write, route: /Topics/026b5e4e-2f92-44b0-8fbf-b717895b12fc
delete False for oak.devices, route: /../../ClientCredentialClients/444d29af-e82e-4224-b2ee-edbc9412115a
update False for oak.omf.write acl, route: /accesscontrol/Types
update False for oak.omf.write acl, route: /accesscontrol/Streams
delete False for oak.omf.write, route: /../../Roles/c1682e0d-9383-4344-ac3a-c344598f341f
delete False for oak.omf.write, route: /../../ClientCredentialClients/efcdc912-fd37-4c00-b055-b4eb3b3ab214
delete False for oak.omf.write.restricted, route: /../../ClientCredentialClients/6f1a331c-74f5-414c-bf16-447abcba8d48
delete False for oak.omf.write.restricted, route: /../../Roles/2114bf6a-2f8d-4f7f-a147-3bb032ac9f0d
fini
